## 1. Carga de datos y limpieza inicial

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
data_dir = 'results/'

In [ ]:
#data = pd.read_csv(data_dir + 'GwasCat_associations.r2020-05-03.parsed.csv.gz')
data = pd.read_table(data_dir + 'gwas_cat.filtrado.tsv.gz')

In [ ]:
data.head()

In [ ]:
data.shape

Seleccionamos aquellas instancias que tienen en la columna `alelo_riesgo` un SNP.

In [ ]:
mask_snps = data.alelo_riesgo.str.match("^rs[0-9]+-[ATCG\?]$")
data = data[mask_snps]
data.shape

Miramos y descartamos valores faltantes

In [ ]:
data.isna().sum()

No nos interesan aquellas instancias que tienen valore faltantes en la columna `OR_or_beta`

In [ ]:
data.dropna(subset = ['OR_or_beta'], inplace = True)
data.shape

In [ ]:
data.isna().sum()

#### ESTO SE PUEDE MEJORAR: tiene que haber una única componente gigante

**Opcional 1**: sacamos aquellos alelos que aparezcan una sola vez. Sirve más que nada para achicar la base de datos.

In [ ]:
if False:
    umbral = 1
    mask_alelos_poca_frecuencia = data.alelo_riesgo.value_counts() <=1
    mask_alelos_poca_frecuencia.head()

    alelos_poca_frecuencia = list(mask_alelos_poca_frecuencia.index[mask_alelos_poca_frecuencia.values])
    print(len(alelos_poca_frecuencia))

    mask_descartables = data.alelo_riesgo.isin(alelos_poca_frecuencia)
    mask_descartables

    print(data.shape)
    data = data[~mask_descartables]
    print(data.shape)

**Opcional 2:** Sacamos aquellos fenotipos que aparecen una sola vez:

In [ ]:
umbral = 1
mask_fenotipos_poca_frecuencia = data.fenotipo.value_counts() <=1
mask_fenotipos_poca_frecuencia.head()

In [ ]:
fenotipos_poca_frecuencia = list(mask_fenotipos_poca_frecuencia.index[mask_fenotipos_poca_frecuencia.values])
print(len(fenotipos_poca_frecuencia))

In [ ]:
mask_descartables = data.fenotipo.isin(fenotipos_poca_frecuencia)
mask_descartables

In [ ]:
print(data.shape)
data = data[~mask_descartables]
print(data.shape)

### 2. Tripletes `alelo_riesgo`, `fenotipo` y `OR_or_beta` - Matriz de utilidad/¿incidencia?

In [ ]:
data_tripletes = data[['alelo_riesgo', 'fenotipo', 'OR_or_beta']]
data_tripletes.shape

In [ ]:
data_tripletes

**Matriz de Utilidad/¿incidencia?**

In [ ]:
n_alelos = data_tripletes.alelo_riesgo.nunique()
n_fenotipos = data_tripletes.fenotipo.nunique()

data_bipartita = pd.DataFrame(np.zeros((n_alelos, n_fenotipos)), columns = data_tripletes.fenotipo.value_counts().index, dtype = np.int8)
data_bipartita.head()

In [ ]:
### EN EL DICCIONARIO APARECEN ORDENADOS POR FRECUENCIA
dict_id_to_alelo = {}
for i,alelo in enumerate(data_tripletes.alelo_riesgo.value_counts().index):
    dict_id_to_alelo[i] = alelo
    
dict_alelo_to_id = {v: k for k, v in dict_id_to_alelo.items()}

In [ ]:
for _, row in data_tripletes.iterrows():
    alelo_row = row.alelo_riesgo
    fenotipo_row = row.fenotipo
    id_alelo = dict_alelo_to_id[alelo_row]
    data_bipartita.loc[id_alelo, fenotipo_row] = 1

**Chequeos**

La suma de todos los elementos de `data_bipartita` debe dar la cantidad de filas en `data_tripletes`

In [ ]:
data_bipartita.sum().sum() == data_tripletes.shape[0]

La suma por columnas el `value_counts()` de `data_tripletes.fenotipo`

In [ ]:
(data_tripletes.fenotipo.value_counts().values == data_bipartita.sum().values).all()

La suma por filas el `value_counts()` de `data_tripletes.alelo_riesgo`

In [ ]:
(data_tripletes.alelo_riesgo.value_counts().values == data_bipartita.sum(axis = 1).values).all()

In [ ]:
data_bipartita

**Distribución de grado para fenotipos**

In [ ]:
plt.hist(np.log(data_bipartita.sum()))

**Distribución de grado para alelos**

In [ ]:
plt.hist(data_bipartita.sum(axis = 1), log=True)

¿VALE LA PENA ESTUDIAR ESTAS DISTRIBUCIONES?

### Separación de datos para testeo

#### Separación 1 - Alelos "nuevos"

Tomamos alelos al azar y los sacamos de la matriz de utilidad. De esta forma, no aportarán a las similitudes de los fenotipos.

Vamos a sacar alelos que tengan al menos grado 3 y como máximo grado 5 

In [ ]:
np.random.seed(42)
n_alelos_a_sacar = 100
alelos_a_sacar = []

lista_alelos_desordenada = list(dict_id_to_alelo.keys())
lista_alelos_desordenada = np.random.choice(lista_alelos_desordenada,len(lista_alelos_desordenada), replace = False)

grados = data_bipartita.sum(axis = 1)
for alelo in lista_alelos_desordenada:
#     pass
    grado = grados.iloc[alelo]
    
    if grado >=3 and grado <=5:
        alelos_a_sacar.append(alelo)
        
    if len(alelos_a_sacar) == n_alelos_a_sacar:
        break

In [ ]:
data_bipartita_test_1 = data_bipartita.iloc[alelos_a_sacar,:]
print(data_bipartita_test_1.shape)
print(data_bipartita_test_1.sum().sum())

In [ ]:
data_bipartita_train = data_bipartita.drop(alelos_a_sacar)
print(data_bipartita_train.shape)

#### ESTO SE PUEDE MEJORAR, PERO POR AHORA CHEQUEAMOS QUE NO QUEDE UN FENOTIPO DE GRADO 1
Nuevamente, lo mejor sería que el conjunto de train tenga una única componente.

In [ ]:
data_bipartita_train.sum().min()

Sacamos del conjunto de test algunos fenotipos conocidos, hasta que quede un solo fenotipo conocido. ¿ESTUDIAR EN FUNCION DE ESTE HIPERPARÁMETRO?

In [ ]:
np.random.seed(42)
n_fenotipos_a_dejar = 1

for idx, row in data_bipartita_test_1.iterrows():
    
    fenotipos_asociados = list(row[row == 1].index)
    fenotipos_asociados = np.random.choice(fenotipos_asociados, len(fenotipos_asociados), replace = False)
    contador = 0
    for fenotipo in fenotipos_asociados:
        data_bipartita_test_1.loc[idx, fenotipo] = 0
        contador +=1
        if contador == len(fenotipos_asociados) - n_fenotipos_a_dejar:
            break

In [ ]:
data_bipartita_test_1.sum().sum()

### 3. Armamos la red --> ¿Pasar a Gephy la visualización?

Se puede saltear por ahora, no lo usamos

In [ ]:
import networkx as nx

In [ ]:
red = nx.Graph()

In [ ]:
red.add_edges_from(data_tripletes[['alelo_riesgo', 'fenotipo']].values)

In [ ]:
### MUY PESADO EN MEMORIA
matriz_adyacencia = nx.to_pandas_adjacency(red, dtype = np.int8)

In [ ]:
# prueba = np.ones((99365, 99365), dtype = np.uint8)
# del prueba

In [ ]:
matriz_adyacencia

In [ ]:
data_bipartita

### 4. Filtro Colaborativo Implícito

https://medium.com/radon-dev/item-item-collaborative-filtering-with-binary-or-unary-data-e8f0b465b2c3

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

En primer lugar, normalizamos con respecto a la cantidad de enlaces que tiene cada alelo. En las películas, esto se justifica como 

> *This is the idea of normalizing the user vectors so that a user with many ratings contributes less to any individual rating. This is to say that a like from a user who has only liked 10 items is more valuable to us than a like from someone who likes everything she comes across.*

In [ ]:
#------------------------
# ITEM-ITEM CALCULATIONS
#------------------------

# As a first step we normalize the user vectors to unit vectors.

# magnitude = sqrt(x2 + y2 + z2 + ...)
magnitude = np.sqrt(np.square(data_bipartita_train).sum(axis=1))

# # unitvector = (x / magnitude, y / magnitude, z / magnitude, ...)
data_bipartita_normalizada = data_bipartita_train.divide(magnitude, axis='index')
data_bipartita_normalizada

Luego,calculamos la similaridad coseno para cada columna, obteniendo así qué fenotipos son parecidos.

In [ ]:
def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

# Build the similarity matrix
data_matrix = calculate_similarity(data_bipartita_normalizada)
data_matrix.head()

Veamos similaridades

In [ ]:
# Lets get the top 11 similar artists for Beyonce
# print(data_matrix.loc['Drinking behavior'].nlargest(25))

print(data_matrix.loc['Educational attainment (MTAG)'].nlargest(30))

Construimos una nueva matriz, con los `n_neighbours` de cada fenotipo. Vendrían a ser los `n_neighbours` fenotipos más cercanos a cada fenotipos.

In [ ]:
n_neighbours = 20
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,n_neighbours+1))
for i in range(0, len(data_matrix.columns)):
    data_neighbours.iloc[i,:n_neighbours] = data_matrix.iloc[0:,i].sort_values(ascending=False)[:n_neighbours].index
data_neighbours.head()

Elegimos un alelo dentro del conjunto de entrenamiento

In [ ]:
# np.random.seed(43)
alelo_prueba = np.random.choice(data_tripletes.alelo_riesgo.unique())
# alelo_prueba = 'rs6739779-C'
print(f'Alelo de prueba: {alelo_prueba}')

alelo_prueba_index = dict_alelo_to_id[alelo_prueba]
print(f'Indice del alelo de prueba: {alelo_prueba_index}')

known_alelo_fenotipos = data_bipartita_normalizada.iloc[alelo_prueba_index]
known_alelo_fenotipos = known_alelo_fenotipos[known_alelo_fenotipos >0].index.values
print(f'Fenotipos asociados al alelo: {known_alelo_fenotipos}')

Dado esos fenotipos, seleccionamos los similares a partir de `data_neighbours`

In [ ]:
# Construct the neighbourhood from the most similar items to the
# ones our alelo it's related
most_similar_fenotipos = data_neighbours.loc[known_alelo_fenotipos]
# most_similar_fenotipos
similar_list = most_similar_fenotipos.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
print(f'Fenotipos similares, sin rankear: {similar_list}')

Seleccionamos, dentro de la matriz de similaridades, los valores para los fenotipos similares

In [ ]:
neighbourhood = data_matrix[similar_list].loc[similar_list]
neighbourhood.shape

Creamos un vector que tiene los fenotipos similares, incluyendo los conocidos, y su *rating* (1 o 0).

In [ ]:
# A user vector containing only the neighbourhood items and
# the known user likes.
fenotipos_probables = data_bipartita_normalizada.iloc[alelo_prueba_index].loc[similar_list]
fenotipos_probables

Calculamos los scores y ordenamos.

In [ ]:
# Calculate the score.
score = neighbourhood.dot(fenotipos_probables).div(neighbourhood.sum(axis=1))
# score
score.sort_values(ascending = False, inplace = True)
# score

E imprimimos los fenotipos conocidos y el ranking

In [ ]:
print(known_alelo_fenotipos)
print(score)

In [ ]:
# sns.barplot(score.index, score.values)
plt.scatter(score.index, score.values)
plt.xticks(rotation = 90)

In [ ]:
def recomendaciones_alelo(alelo_index, data_bipartita_normalizada, data_neighbours, data_matrix):
    known_alelo_fenotipos = data_bipartita_normalizada.iloc[alelo_index]
    known_alelo_fenotipos = known_alelo_fenotipos[known_alelo_fenotipos >0].index.values

    most_similar_fenotipos = data_neighbours.loc[known_alelo_fenotipos]

    similar_list = most_similar_fenotipos.values.tolist()
    similar_list = list(set([item for sublist in similar_list for item in sublist]))

    neighbourhood = data_matrix[similar_list].loc[similar_list]

    fenotipos_probables = data_bipartita_normalizada.iloc[alelo_prueba_index].loc[similar_list]

    score = neighbourhood.dot(fenotipos_probables).div(neighbourhood.sum(axis=1))
    score.sort_values(ascending = False, inplace = True)

    return score

In [ ]:
score = recomendaciones_alelo(alelo_prueba_index, data_bipartita_normalizada, data_neighbours, data_matrix)

In [ ]:
plt.scatter(score.index, score.values)
plt.xticks(rotation = 90)

### Sobre Conjunto de Test

"A mano" por ahora

In [ ]:
# np.random.seed(50)
alelo_prueba_index = np.random.choice(data_bipartita_test_1.index)
print(f'Indice alelo a probar: {alelo_prueba_index}')

alelo_prueba = dict_id_to_alelo[alelo_prueba_index]
print(f'Alelo a probar: {alelo_prueba}')

known_alelo_fenotipos = data_bipartita_test_1.loc[alelo_prueba_index]
known_alelo_fenotipos = known_alelo_fenotipos[known_alelo_fenotipos >0].index.values
print(f'Fenotipos asociados al alelo para generar recomendaciones: {known_alelo_fenotipos}')

In [ ]:
fenotipos_reportados_alelo = data_bipartita.loc[alelo_prueba_index]
fenotipos_reportados_alelo = fenotipos_reportados_alelo[fenotipos_reportados_alelo >0].index.values
print(f'Todos fenotipos asociados al alelo: {fenotipos_reportados_alelo}')

In [ ]:
most_similar_fenotipos = data_neighbours.loc[known_alelo_fenotipos]
similar_list = most_similar_fenotipos.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
neighbourhood = data_matrix[similar_list].loc[similar_list]

In [ ]:
fenotipos_probables = data_bipartita_test_1.loc[alelo_prueba_index].loc[similar_list]
# fenotipos_probables

In [ ]:
score = neighbourhood.dot(fenotipos_probables).div(neighbourhood.sum(axis=1))
score.sort_values(ascending = False, inplace = True)
print(known_alelo_fenotipos)
print(fenotipos_reportados_alelo)
print(score.nlargest(50))

In [ ]:
plt.scatter(score.index, score.values)
plt.xticks(rotation = 90)

1. grafico de fenotipos vs score
1. capacidad de priorizar dado 
1. Usar el paper de zhou


MARTES - 5 min
dimensión de los datos
encuadrar el problema biológico o metodológico



In [ ]:
def indices_aciertos(fenotipos_recomendados, fenotipos_a_evaluar):
    indices = []
    for fenotipo in fenotipos_a_evaluar:
        if fenotipo in fenotipos_recomendados:
            indices.append(fenotipos_recomendados.index(fenotipo))
        else:
            indices.append(-1)
    return indices

In [ ]:
n = 10
fenotipos_recomendados = list(score.index)
fenotipos_usados_para_recomendar = known_alelo_fenotipos
fenotipos_conocidos = fenotipos_reportados_alelo

n_0 = len(fenotipos_usados_para_recomendar)
fenotipos_recomendados = fenotipos_recomendados[n_0:n_0+n]
fenotipos_a_evaluar = [x for x in fenotipos_conocidos if x not in fenotipos_usados_para_recomendar]

indices_aciertos(fenotipos_recomendados, fenotipos_a_evaluar)

In [ ]:
n = 10

resultados = []

for alelo_prueba_index in data_bipartita_test_1.index:
#     print(alelo_prueba_index)
    alelo_prueba = dict_id_to_alelo[alelo_prueba_index]
    known_alelo_fenotipos = data_bipartita_test_1.loc[alelo_prueba_index]
    known_alelo_fenotipos = known_alelo_fenotipos[known_alelo_fenotipos >0].index.values
#     print(f'Fenotipos asociados al alelo para generar recomendaciones: {known_alelo_fenotipos}')
    
    fenotipos_reportados_alelo = data_bipartita.loc[alelo_prueba_index]
    fenotipos_reportados_alelo = fenotipos_reportados_alelo[fenotipos_reportados_alelo >0].index.values
#     print(f'Todos fenotipos asociados al alelo: {fenotipos_reportados_alelo}')
    
    most_similar_fenotipos = data_neighbours.loc[known_alelo_fenotipos]
    similar_list = most_similar_fenotipos.values.tolist()
    similar_list = list(set([item for sublist in similar_list for item in sublist]))
    neighbourhood = data_matrix[similar_list].loc[similar_list]
    
    fenotipos_probables = data_bipartita_test_1.loc[alelo_prueba_index].loc[similar_list]
    score = neighbourhood.dot(fenotipos_probables).div(neighbourhood.sum(axis=1))
    score.sort_values(ascending = False, inplace = True)
#     print(score.nlargest(5))

    
    fenotipos_recomendados = list(score.index)
    fenotipos_usados_para_recomendar = known_alelo_fenotipos
    fenotipos_conocidos = fenotipos_reportados_alelo
    
    n_0 = len(fenotipos_usados_para_recomendar)
    fenotipos_recomendados = fenotipos_recomendados[n_0:n_0+n]
    fenotipos_a_evaluar = [x for x in fenotipos_conocidos if x not in fenotipos_usados_para_recomendar]
    
    idxs = indices_aciertos(fenotipos_recomendados, fenotipos_a_evaluar)
    
    resultados_alelo = dict(zip(fenotipos_a_evaluar, idxs))
    resultados_alelo['alelo'] = alelo_prueba
    resultados_alelo['indice_alelo'] = alelo_prueba_index
    resultados.append(resultados_alelo)
    
#     print(idxs)


In [ ]:
resultados_df = pd.DataFrame(resultados).set_index('indice_alelo')
resultados_df.head()

In [ ]:
indices = resultados_df.drop(columns = 'alelo').values
indices = indices[~np.isnan(indices)]
plt.hist(indices, bins = np.arange(-1.5,9.5))

In [ ]:
print(f'Porcentaje aciertos: {(indices>-1).sum()/indices.size}')
print(f'Porcentaje no aciertos: {(indices==-1).sum()/indices.size}')

### Cosas para hacer

1. Chequear sobre `categoria_fenotipo` que no sea trivial
1. Agregar segunda forma de evaluación: borrando '1' de la matriz de train
1. Chequear que al separar train y test no se rompa la red
1. Metodizar la evaluación para obtener una métrica
1. Chequear que no estemos cayendo en la parte "fácil" de la red --> ¿pregunta biológica?